# Домашнее задание 2 по обработке текстов

Рассмотрим задачу бинарной классификации. Пусть дано два списка имен: мужские и женские имена. Требуется разработать классификатор, который по данному имени будет определять мужское оно или женское.

Данные: 
* Женские имена: female.txt
* Мужские имена: male.txt

## Часть 1. Предварительная обработка данных

1. Удалите неоднозначные имена (те имена, которые являются и мужскими, и женскими дновременно), если такие есть; 
2. Создайте обучающее и тестовое множество так, чтобы в обучающем множестве классы были сбалансированы, т.е. к классу принадлежало бы одинаковое количество имен;

In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
male = pd.read_csv("male.txt")

In [3]:
male.columns = ["Name"]
male.head()

,Name
0,Aaron
1,Abbey
2,Abbie
3,Abbot
4,Abbott


In [4]:
male.columns

Index(['Name'], dtype='object')

In [5]:
female = pd.read_csv("female.txt")

In [6]:
female.columns = ["Name"]
female.head()

,Name
0,Abagail
1,Abbe
2,Abbey
3,Abbi
4,Abbie


In [7]:
male['isMale'] = 1

In [8]:
female['isMale'] = 0

In [9]:
m_f = male.append(female)

In [10]:
m_f.Name = m_f.Name.str.lower()

In [11]:
m_f.drop_duplicates(subset=["Name"], inplace=True)

In [12]:
m_f

,Name,isMale
0,aaron,1
1,abbey,1
2,abbie,1
3,abbot,1
4,abbott,1
5,abby,1
6,abdel,1
7,abdul,1
8,abdulkarim,1
9,abdullah,1


##  Часть 2. Базовый метод классификации

Используйте метод наивного Байеса или логистическую регрессию для классификации имен: в качестве признаков используйте символьные $n$-граммы. Сравните результаты, получаемые при разных $n=2,3,4$ по $F$-мере и аккуратности. В каких случаях метод ошибается?

Для генерации $n$-грамм используйте:

In [13]:
import nltk

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer

In [15]:
X_train, X_test, y_train, y_test = train_test_split(m_f.Name, m_f.isMale, test_size=0.2, random_state = 715)

In [16]:
clf_1 = MultinomialNB(alpha = 0.5)

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(ngram_range=(1, 5),max_features = 7000, analyzer='char_wb')

In [18]:
X_train_v = count_vect.fit_transform(X_train)

In [19]:
X_train_v

<6059x7000 sparse matrix of type '<class 'numpy.int64'>'
	with 147410 stored elements in Compressed Sparse Row format>

In [20]:
X_test_v = count_vect.fit_transform(X_test)

In [21]:
X_test_v

<1515x7000 sparse matrix of type '<class 'numpy.int64'>'
	with 38818 stored elements in Compressed Sparse Row format>

In [22]:
clf_1.fit(X_train_v, y_train)

MultinomialNB(alpha=0.5, class_prior=None, fit_prior=True)

In [23]:
pred = clf_1.predict(X_test_v)

In [24]:
accuracy_score(y_test, pred)

0.55841584158415847

In [25]:
clf_2 = LogisticRegression()

In [26]:
clf_2.fit(X_train_v, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [27]:
pred_2 = clf_2.predict(X_test_v)

In [28]:
accuracy_score(y_test, pred_2)

0.45346534653465348

Попробуем улучшить классификатор, показавший наилучший результат:

In [29]:
from sklearn.model_selection import GridSearchCV

In [30]:
clf = Pipeline([
        ('v', CountVectorizer(analyzer='char_wb')),    
        ('clf', MultinomialNB())])

In [37]:
params = {'v__ngram_range': [(1, 5)], 'clf__alpha': (0.001, 0.01, 0.1, 0.5, 1, 2)}

In [38]:
GridSearch = GridSearchCV(clf, params, scoring='accuracy', cv=5, n_jobs=-1)

In [39]:
GridSearch.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(steps=[('v', CountVectorizer(analyzer='char_wb', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'v__ngram_range': [(1, 5)], 'clf__alpha': (0.001, 0.01, 0.1, 0.5, 1, 2)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=0)

In [44]:
print (GridSearch.best_params_)
print (GridSearch.best_estimator_)
print (GridSearch.best_score_)

{'clf__alpha': 0.1, 'v__ngram_range': (1, 5)}
Pipeline(steps=[('v', CountVectorizer(analyzer='char_wb', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 5), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)), ('clf', MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True))])
0.853606205644


In [45]:
preds = GridSearch.best_estimator_.predict(X_test)

In [46]:
print("F1-score for best estimator: {0:6.2f}".format(f1_score(y_test, preds)))
print("Accuracy for best estimator: {0:6.2f}\n".format(accuracy_score(y_test, preds)))

F1-score for best estimator:   0.83
Accuracy for best estimator:   0.87



* F1-score for 1-gram:   0.51 | Accuracy for 1-gram:   0.71

* F1-score for 2-gram:   0.77 | Accuracy for 2-gram:   0.83

* F1-score for 3-gram:   0.83 | Accuracy for 3-gram:   0.87

* F1-score for 4-gram:   0.80 | Accuracy for 4-gram:   0.86

* F1-score for 5-gram:   0.70 | Accuracy for 5-gram:   0.81

Итак, мы видим, что с увеличением n качество предсказаний растет, но у данной тенденции есть предел и начиная с n=3 качество начинает ухудшаться, по всей видимости, модель переобучается.

##  Часть 3. Нейронная сеть


Используйте  реккурентную нейронную сеть с  LSTM для решения задачи. В ней может быть несколько слоев с LSTM, несколько слоев c Bidirectional(LSTM).  У нейронной сети один выход, определяющий класс имени. 

Представление имени для классификации в этом случае: бинарная матрица размера (количество букв в алфавите $\times$ максимальная длина имени). Обозначим его через $x$. Если первая буква имени a, то $x[1][1] = 1$, если вторая – b, то  $x[2][1] = 1$.  

Не забудьте про регуляризацию нейронной сети дропаутами. 

Сравните результаты классификации разными методами. Какой метод лучше и почему?

Сравните результаты, получаемые при разных значениях дропаута, разных числах узлов на слоях нейронной сети по $F$-мере и аккуратности. В каких случаях нейронная сеть ошибается?

Если совсем не получается запрограммировать нейронную сеть самостоятельно, обратитесь к туториалу тут: https://github.com/divamgupta/lstm-gender-predictor